In [19]:
import torch
import numpy as np
import torchvision
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [20]:
n_train,n_test,true_w,true_b=100,100,[1.2,-3.4,5.6],5
features=torch.randn((n_train+n_test,1))

In [21]:
poly_features = torch.cat(
    (features, torch.pow(features, 2), torch.pow(features, 3)), 1)
labels = (true_w[0] * poly_features[:, 0] + true_w[1] * poly_features[:, 1] +
          true_w[2] * poly_features[:, 2] + true_b)
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                       dtype=torch.float)

In [22]:
poly_features

tensor([[-1.2812e+00,  1.6416e+00, -2.1032e+00],
        [ 1.7959e+00,  3.2253e+00,  5.7924e+00],
        [ 3.2343e-01,  1.0460e-01,  3.3832e-02],
        [ 1.0292e-01,  1.0592e-02,  1.0901e-03],
        [ 1.5740e+00,  2.4773e+00,  3.8992e+00],
        [-1.0276e-01,  1.0560e-02, -1.0852e-03],
        [ 1.3081e+00,  1.7112e+00,  2.2384e+00],
        [-1.4347e+00,  2.0583e+00, -2.9529e+00],
        [ 1.2112e+00,  1.4670e+00,  1.7769e+00],
        [ 2.8427e-01,  8.0810e-02,  2.2972e-02],
        [-1.4046e+00,  1.9728e+00, -2.7709e+00],
        [-2.2276e-01,  4.9622e-02, -1.1054e-02],
        [-6.2926e-01,  3.9597e-01, -2.4917e-01],
        [-6.7308e-01,  4.5304e-01, -3.0493e-01],
        [-2.6142e-01,  6.8340e-02, -1.7866e-02],
        [-1.2746e+00,  1.6246e+00, -2.0707e+00],
        [ 1.0422e+00,  1.0862e+00,  1.1320e+00],
        [-7.0805e-01,  5.0134e-01, -3.5497e-01],
        [-2.6656e-01,  7.1055e-02, -1.8940e-02],
        [ 1.3325e+00,  1.7755e+00,  2.3657e+00],
        [ 9.7219e-01

In [29]:
def semilogy(x_vals,y_vals,x_label,y_label,x2_vals=None,y2_vals=None,
            legend=None,figsize=(3.5,2.5)):
    plt.figure(figsize=figsize)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.semilogx(x_vals,y_vals)
    if x2_vals and y2_vals:
        plt.semilogy(x2_vals,y2_vals,linestyle=':')
        plt.legend(legend)

In [30]:
num_epochs,loss=100,torch.nn.MSELoss()
def fit_and_plot(train_features,test_features,train_labels,test_labels):
    net=torch.nn.Linear(train_features.shape[-1],1)
    batch_size=min(10,train_labels.shape[0])
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    
    optimizer=torch.optim.SGD(net.parameters(),lr=0.01)
    train_ls,test_ls=[],[]
    for _ in range(num_epochs):
        for x,y in train_iter:
            l=loss(net(x),y.view(-1,1))
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_labels=train_labels.view(-1,1)
            test_labels=test_labels.view(-1,1)
            train_ls.append(loss(net(train_features),train_labels).item())
            test_ls.append(loss(net(test_features),test_labels).item())
        print('final epoch:train loss',train_ls[-1],'test loss',test_ls[-1])
        semilogy(range(1,num_epochs+1),train_ls,'epochs','loss',
                range(1,num_epochs+1),test_ls,['train','test'])
        print('weight: ',net.weight.data,'\nbias:',net.bias.data)
        

In [33]:
fit_and_plot(poly_features[:n_train,:],poly_features[n_train:,:],
            labels[:n_train],labels[n_train:])

final epoch:train loss 23.440837860107422 test loss 21.59218978881836


ValueError: x and y must have same first dimension, but have shapes (100,) and (10,)